In [6]:
import os
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [7]:
google_api_key=os.getenv("GOOGLE_API_KEY")

In [8]:
if google_api_key=="":
    print("API key not found.")
else:
    print("API key found.")

API key found.


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.gemini import Gemini
from IPython.display import Markdown, display
from llama_index.core import ServiceContext
from llama_index.core import StorageContext, load_index_from_storage
import google.generativeai as genai
from llama_index.embeddings.gemini import GeminiEmbedding

In [10]:
genai.configure(api_key=google_api_key)

In [11]:
for models in genai.list_models():
    print(models)

Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      max_temperature=None,
      top_p=0.95,
      top_k=40)
Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      max_temperature=None,
      top_p=0.95,
      top_k=40)
Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko

In [12]:
for models in genai.list_models():
    if 'generateContent' in models.supported_generation_methods:
        print(models.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-think

In [15]:
documents=SimpleDirectoryReader("../Data")

In [17]:
doc=documents.load_data()

In [20]:
print(doc[0].text)

Alapuzha Gymkhana(2025) Plot:
Jojo Johnson, David "DJ" John, Shifas "Valthu" Ahammed, Shifas "Cherthu" Ali, and Shanavas are five friends from Alappuzha who have just completed their 12th-grade board exams. While Shanavas passes, the rest of the group fails, leaving their prospects of college admission uncertain.

After Valthu gets into a scuffle for flirting with a state-level boxer’s girlfriend, Jojo begins researching alternative ways to enter college and discovers that sports quota admissions through boxing tournaments are a viable route. The group decides to pursue boxing and enroll at the local Alappuzha Gymkhana boxing school, under coach Salim. However, Shanavas soon loses interest and quits, while DJ switches to wrestling.

Jojo, Valthu, and Cherthu persist and are later mentored by Antony Joshua, a former student of Salim and a current state-level boxer training for the South Asia Championship. Antony enforces a strict training and diet plan, pushing them to their physical li

In [21]:
model=Gemini(models='gemini-pro',api_key=google_api_key)

C:\Users\visha\AppData\Local\Temp\ipykernel_20036\2941566521.py:1: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  model=Gemini(models='gemini-pro',api_key=google_api_key)


In [22]:
gemini_embed_model=GeminiEmbedding(model_name="models/embedding-001")

C:\Users\visha\AppData\Local\Temp\ipykernel_20036\4148756438.py:1: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  gemini_embed_model=GeminiEmbedding(model_name="models/embedding-001")


In [25]:
# Configure Service Context
from llama_index.core import Settings

# Set the default models and parameters globally
Settings.llm = model
Settings.embed_model = gemini_embed_model
Settings.chunk_size = 800
Settings.chunk_overlap = 20

In [27]:
index = VectorStoreIndex.from_documents(doc)

In [28]:
index.storage_context.persist()

In [29]:
query_engine=index.as_query_engine()

In [31]:
response=query_engine.query("What is the plot about?")

In [33]:
print(response.response)

Five friends in Alappuzha, having just finished their 12th-grade exams, face uncertain futures when only one of them passes. They discover that sports quota admissions through boxing tournaments could be a viable path to college. They decide to pursue boxing, joining the local Alappuzha Gymkhana boxing school. They face challenges, including strict training, district-level championships, and unexpected events at the state-level championship, where they encounter setbacks and external threats. Despite the difficulties, they decide to train harder and return the following year, with one friend taking on the role of team manager and two others preparing to join the team.



In [34]:
response=query_engine.query("Who is protagonist in this movie?")

In [35]:
print(response.response)

Jojo Johnson is one of the protagonists in this movie.



In [36]:
response=query_engine.query("Who is Muhammad Ali")

In [37]:
print(response.response)

The context does not contain information about Muhammad Ali.

